In [1]:
import sys
import os
from PIL import Image
import cv2

In [ ]:
focal_length = 800.0
scaling_factor = 2000.0

In [14]:
def generate_point_cloud(image_path, depth_image_path, output_ply_path):
    rgb = Image.open(image_path).convert("RGB")
    depth = Image.open(depth_image_path).convert("I")
    depth = depth.resize(rgb.size)
    
    centerX = rgb.width / 2
    centerY = rgb.height / 2
    
    if rgb.size != depth.size:
        print("Error: RGB and depth image sizes do not match.")
        return
    if rgb.mode != "RGB":
        print("Error: RGB image is not in RGB mode.")
        return
    if depth.mode != "I":
        print("Error: Depth image is not in 32-bit integer mode.")
        return
    
    points = []
    for v in range(rgb.height):
        for u in range(rgb.width):
            color = rgb.getpixel((u, v))
            Z = depth.getpixel((u, v)) / scaling_factor
            if Z == 0:
                continue
            X = (u - centerX) * Z / focal_length
            Y = (v - centerY) * Z / focal_length            
            points.append(f"{X} {Y} {Z} {color[0]} {color[1]} {color[2]}\n")
            
    with open(output_ply_path, 'w') as f:
        f.write(f"ply\nformat ascii 1.0\nelement vertex {len(points)}\n")
        f.write("property float x\nproperty float y\nproperty float z\n")
        f.write("property uchar red\nproperty uchar green\nproperty uchar blue\nend_header\n")
        f.writelines(points)
    

In [15]:
rgb_image_path = "mountain_color.png"
depth_image_path = "mountain_depth.png"
output_ply_path = "result.ply"

generate_point_cloud(rgb_image_path, depth_image_path, output_ply_path)